<a href="https://colab.research.google.com/github/ihrisikesa/Adjudication_App/blob/main/Adjudication_Kobo_App_INC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# # Intranatal Care Adjudication — BigQuery-backed (Schema auto-read from BQ)
# - Reads column list from BigQuery table schema (no CSV)
# - Saves to BigQuery tables: adj_inc_long, adj_inc_wide, adj_inc_progress, adj_inc_summary
# - Finalize Case (mark done) + Reopen, with progress persistence
# - Comparison/Adjudication toggles; strict gating; fast interactions


# ## 0) Install & Imports
!pip -q install google-cloud-bigquery google-auth pandas pyarrow numpy ipywidgets matplotlib-venn


import re, time, random
from typing import List, Dict, Tuple
from datetime import datetime, timezone
from IPython.display import HTML

import matplotlib.pyplot as plt
from matplotlib_venn import venn2

import numpy as np
import pandas as pd

# Colab auth
try:
    from google.colab import auth
except Exception:
    auth = None

import google.auth
from google.cloud import bigquery

import ipywidgets as W
from IPython.display import display, HTML


# ## 1) CONFIG — EDIT THESE

# BigQuery
PROJECT_ID   = ""   # e.g., ""
DATASET_ID   = ""       # e.g., ""
TABLE_ID     = ""  # change if needed

# Staff IDs to compare (strings)
STAFF_A = ""
STAFF_B = ""
ADJUDICATOR_NAME  = ""


# Columns that uniquely identify a record (case)
KEY_COLUMNS   = ["case_id"]      # e.g. ["patient_id","tanggal"]
TIMESTAMP_COL = None        # e.g. "updated_at" -> keep latest per staff

# Matching behavior
NULL_EQUALS_MATCH = True
ROUND_DIGITS      = 6

# UI defaults
MAX_TABLE_HEIGHT_PX   = 480
HIDE_COMPLETED_DEFAULT= True
SAVE_ON_FINALIZE      = True

# Fields to exclude from adjudication (exact, case-insensitive)
EXCLUDE_USER_FIELDS = {
    "field","deviceid","start","end","username","_id","_uuid","_submission_time","_validation_status",
    "_notes","_status","_submitted_by","__version__","_tags","_index"
}

# Destination tables (must already exist)
TBL_LONG     = "adj_inc_long"
TBL_WIDE     = "adj_inc_wide"
TBL_PROGRESS = "adj_inc_progress"
TBL_SUMMARY  = "adj_inc_summary"

# Only allow scalar types from source schema
ALLOWED_TYPES = {
    "STRING","BOOL","BOOLEAN","INT64","FLOAT64","NUMERIC","BIGNUMERIC",
    "DATE","DATETIME","TIME","TIMESTAMP"
}


# ## 2) Authenticate to BigQuery
if auth is not None:
    auth.authenticate_user()
creds, _ = google.auth.default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
bq = bigquery.Client(project=PROJECT_ID, credentials=creds)
print("✅ BigQuery authenticated.\n")


def plot_venn_cases_A_B():
    """
    Venn diagram of unique cases by STAFF_A vs STAFF_B.
    Uses KEY_COLUMNS as the case key (tuple if multiple).
    Excludes rows with NULL in any key column.
    """
    # Build expressions
    key_select = ", ".join([wrap_tick(k) for k in KEY_COLUMNS])
    not_null_cond = " AND ".join([f"{wrap_tick(k)} IS NOT NULL" for k in KEY_COLUMNS])

    sql = f"""
    SELECT
      {key_select},
      COUNTIF({wrap_tick(STAFF_COL)} = @staffA) AS cA,
      COUNTIF({wrap_tick(STAFF_COL)} = @staffB) AS cB
    FROM `{TABLE_FQN}`
    WHERE {not_null_cond}
    GROUP BY {key_select}
    """

    job_cfg = bigquery.QueryJobConfig(query_parameters=[
        bigquery.ScalarQueryParameter("staffA", "STRING", STAFF_A),
        bigquery.ScalarQueryParameter("staffB", "STRING", STAFF_B),
    ])

    df = bq.query(sql, job_config=job_cfg).to_dataframe(create_bqstorage_client=False)

    set_A = set()
    set_B = set()

    for _, r in df.iterrows():
        # build key tuple (works for single or multiple KEY_COLUMNS)
        key = tuple(str(r[k]) for k in KEY_COLUMNS)
        if r["cA"] > 0:
            set_A.add(key)
        if r["cB"] > 0:
            set_B.add(key)

    # Draw Venn
    plt.figure(figsize=(4,4))
    v = venn2(
        subsets=(set_A, set_B),
        set_labels=(f"A: {STAFF_A}", f"B: {STAFF_B}")
    )
    plt.title("Unique cases by staff (A vs B)")
    plt.show()

    # Print counts for clarity
    only_A = len(set_A - set_B)
    only_B = len(set_B - set_A)
    both   = len(set_A & set_B)
    union  = len(set_A | set_B)

   # print(f"Unique cases by A        : {len(set_A)}")
   # print(f"Unique cases by B        : {len(set_B)}")
   # print(f"Unique cases A or B      : {union}")
   # print(f"Unique cases A and B     : {both}")
   # print(f"Only A                   : {only_A}")
   # print(f"Only B                   : {only_B}")

# Call it
plot_venn_cases_A_B()



# ## 3) Utilities — table existence, temp loads, MERGE upserts (no CREATE)
def now_utc():
    return datetime.now(timezone.utc)


def bq_schema_map(project, dataset, table):
    tbl = bq.get_table(f"{project}.{dataset}.{table}")
    return {f.name: f.field_type.upper() for f in tbl.schema}

def fq(table_name: str) -> str:
    return f"`{PROJECT_ID}.{DATASET_ID}.{table_name}`"

def assert_tables_exist():
    needed = [TBL_LONG, TBL_WIDE, TBL_PROGRESS, TBL_SUMMARY]
    missing = []
    for t in needed:
        try:
            bq.get_table(f"{PROJECT_ID}.{DATASET_ID}.{t}")
        except Exception:
            missing.append(t)
    if missing:
        raise SystemExit(
            "❌ These destination tables do not exist:\n  - "
            + "\n  - ".join(missing)
            + "\nPlease create them first (schema compatible with previous workflow)."
        )

def existing_columns(table_name: str) -> set:
    tbl = bq.get_table(f"{PROJECT_ID}.{DATASET_ID}.{table_name}")
    return {f.name for f in tbl.schema}

def _tmp_name(base: str) -> str:
    return f"__tmp_{base}_{int(time.time())}_{random.randint(10_000,99_999)}"

def _load_df(df: pd.DataFrame, table_name: str, write="WRITE_TRUNCATE"):
    job_config = bigquery.LoadJobConfig(
        write_disposition=write,
        autodetect=True
    )
    bq.load_table_from_dataframe(df, f"{PROJECT_ID}.{DATASET_ID}.{table_name}", job_config=job_config).result()

def _merge_from_temp(temp_table: str, dest_table: str, key_cols: List[str], update_cols: List[str]):
    on_clause = " AND ".join([f"T.{c}=S.{c}" for c in key_cols])
    all_cols  = key_cols + update_cols
    insert_cols = ", ".join(all_cols)
    insert_vals = ", ".join([f"S.{c}" for c in all_cols])

    if update_cols:
        set_clause = ", ".join([f"{c}=S.{c}" for c in update_cols])
        sql = f"""
        MERGE {fq(dest_table)} T
        USING {fq(temp_table)} S
        ON {on_clause}
        WHEN MATCHED THEN UPDATE SET {set_clause}
        WHEN NOT MATCHED THEN INSERT ({insert_cols}) VALUES ({insert_vals})
        """
    else:
        # No updatable columns -> insert-only path
        sql = f"""
        MERGE {fq(dest_table)} T
        USING {fq(temp_table)} S
        ON {on_clause}
        WHEN NOT MATCHED THEN INSERT ({insert_cols}) VALUES ({insert_vals})
        """
    bq.query(sql).result()
    bq.delete_table(f"{PROJECT_ID}.{DATASET_ID}.{temp_table}", not_found_ok=True)

# Build a cache of the destination schema so we know expected types
PROGRESS_TYPES = bq_schema_map(PROJECT_ID, DATASET_ID, TBL_PROGRESS)

def upsert_progress_row(rec: dict):
    """
    Upsert a single progress record to adj_inc_progress, coercing types to match destination schema.
    - If destination 'decided' is BOOL, we store (decided == required)  [is_complete]
    - If destination 'decided' is INT64, we store the numeric decided count
    - We only include columns that already exist on the destination table
    """
    dest_cols  = existing_columns(TBL_PROGRESS)
    dest_types = PROGRESS_TYPES

    # Pull counts from 'rec' to derive booleans when needed
    dec = rec.get("decided")
    req = rec.get("required")
    is_complete = (dec is not None and req is not None and dec == req)

    # Build output record respecting destination schema
    out = {}
    for col in dest_cols:
        typ = dest_types.get(col)

        # Choose a value for this column
        if col in rec:
            val = rec[col]
        elif col == "decided_count" and dec is not None:
            val = dec
        elif col == "required_count" and req is not None:
            val = req
        elif col in ("is_complete", "complete", "completed"):
            val = is_complete
        else:
            # leave out columns we cannot fill
            continue

        # Coerce by destination type
        if val is None:
            out[col] = None
        elif typ in ("TIMESTAMP",):
            out[col] = pd.to_datetime(val, utc=True)
        elif typ in ("BOOL", "BOOLEAN"):
            if col == "decided":
                out[col] = bool(is_complete)  # store completeness
            else:
                out[col] = bool(val)
        elif typ in ("INT64", "INTEGER"):
            out[col] = int(val)
        else:
            out[col] = str(val)

    # Ensure key columns + case_label are present (as strings) if they exist on destination
    for k in (KEY_COLUMNS + ["case_label"]):
        if k in dest_cols:
            if k == "case_label":
                # If not provided in rec, synthesize from keys
                if "case_label" not in out:
                    # make a label if possible
                    try:
                        idx = CASE_INDEX[rec["case_label"]]
                        out["case_label"] = rec["case_label"]
                    except Exception:
                        # last resort: join key values if available in rec
                        out["case_label"] = rec.get("case_label", "")
            else:
                # copy from rec if present
                if k in rec and k not in out:
                    out[k] = str(rec[k])

    # Prepare DataFrame with proper dtypes (so the temp table column types match)
    df = pd.DataFrame([out])

    for c, t in dest_types.items():
        if c in df.columns:
            if t in ("BOOL", "BOOLEAN"):
                df[c] = df[c].astype("boolean")
            elif t in ("INT64", "INTEGER"):
                df[c] = df[c].astype("Int64")  # nullable integer dtype
            elif t == "TIMESTAMP":
                df[c] = pd.to_datetime(df[c], utc=True)

    tmp = _tmp_name("progress")
    _load_df(df, tmp, write="WRITE_TRUNCATE")

    key_cols   = [c for c in (KEY_COLUMNS + ["case_label"]) if c in dest_cols]
    update_cols= [c for c in df.columns if c not in key_cols]
    _merge_from_temp(tmp, TBL_PROGRESS, key_cols, update_cols)


def write_long_df_upsert(df: pd.DataFrame):
    if df.empty: return
    df = df.copy()
    for c in KEY_COLUMNS + ["field","A_value","B_value","choice","final_value","case_label","staff_A","staff_B"]:
        if c in df.columns: df[c] = df[c].astype(str)
    tmp = _tmp_name("long")
    _load_df(df, tmp)
    dest_cols = existing_columns(TBL_LONG)
    key_cols  = [c for c in (KEY_COLUMNS + ["field"]) if c in dest_cols]
    update_cols = [c for c in ["A_value","B_value","choice","final_value","case_label","staff_A","staff_B","generated_at"] if c in dest_cols]
    _merge_from_temp(tmp, TBL_LONG, key_cols, update_cols)

def write_wide_df_upsert(df: pd.DataFrame):
    if df.empty: return
    df = df.copy()
    for c in KEY_COLUMNS + ["staff_A","staff_B"]:
        if c in df.columns: df[c] = df[c].astype(str)
    tmp = _tmp_name("wide")
    _load_df(df, tmp)
    dest_cols = existing_columns(TBL_WIDE)
    key_cols  = [c for c in KEY_COLUMNS if c in dest_cols]
    # Only update columns that actually exist in destination
    update_cols = [c for c in df.columns if c not in key_cols and c in dest_cols]
    _merge_from_temp(tmp, TBL_WIDE, key_cols, update_cols)

def write_summary_append(df: pd.DataFrame):
    if df.empty: return
    # Append without merge (simple log)
    job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND", autodetect=True)
    bq.load_table_from_dataframe(df, f"{PROJECT_ID}.{DATASET_ID}.{TBL_SUMMARY}", job_config=job_config).result()

assert_tables_exist()


# ## 4) Read field list from BigQuery schema (skip nested/repeated & excluded)
STAFF_COL = "data_entry_clerk_staff_id"
EXCLUDE_EXACT = set([STAFF_COL.lower()]) | set([c.lower() for c in KEY_COLUMNS]) | set([c.lower() for c in EXCLUDE_USER_FIELDS])
EXCLUDE_REGEX = re.compile(r"(created|updated|inserted|modified).*(_at|time|timestamp)$|(^created|^updated|^inserted|^modified)$")

def read_fields_from_bq_table(client: bigquery.Client,
                              project: str, dataset: str, table: str,
                              allowed_types=ALLOWED_TYPES) -> List[str]:
    tbl = client.get_table(f"{project}.{dataset}.{table}")
    cols = []
    for f in tbl.schema:
        if f.field_type.upper() == "RECORD":      # skip nested
            continue
        if str(f.mode).upper() == "REPEATED":     # skip arrays
            continue
        if allowed_types and f.field_type.upper() not in allowed_types:
            continue
        cols.append(f.name)
    return cols

ALL_COLUMNS_FROM_SCHEMA = read_fields_from_bq_table(bq, PROJECT_ID, DATASET_ID, TABLE_ID)

def choose_compare_fields(all_cols: List[str]) -> List[str]:
    keep = []
    for c in all_cols:
        cl = c.lower()
        if cl in EXCLUDE_EXACT:        continue
        if TIMESTAMP_COL and cl == TIMESTAMP_COL.lower(): continue
        if EXCLUDE_REGEX.search(cl):   continue
        keep.append(c)
    return keep

FIELDS = choose_compare_fields(ALL_COLUMNS_FROM_SCHEMA)
if not FIELDS:
    raise SystemExit("No eligible fields found from BigQuery schema after exclusions. Check CONFIG/exclusions.")
print(f"📋 Will compare {len(FIELDS)} fields.")


# ## 5) Load source rows for both staff → pivot A/B
def wrap_tick(col: str) -> str: return f"`{col}`"

def build_select_list(keys: List[str], fields: List[str], include_timestamp: bool) -> str:
    parts = [wrap_tick(k) for k in keys] + [wrap_tick(STAFF_COL)] + [wrap_tick(f) for f in fields]
    if include_timestamp and TIMESTAMP_COL:
        parts.append(wrap_tick(TIMESTAMP_COL))
    return ", ".join(parts)

TABLE_FQN = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"
select_list = build_select_list(KEY_COLUMNS, FIELDS, include_timestamp=bool(TIMESTAMP_COL))

sql = f"""
SELECT {select_list}
FROM `{TABLE_FQN}`
WHERE {wrap_tick(STAFF_COL)} IN (@staffA, @staffB)
"""
job_config = bigquery.QueryJobConfig(query_parameters=[
    bigquery.ScalarQueryParameter("staffA", "STRING", STAFF_A),
    bigquery.ScalarQueryParameter("staffB", "STRING", STAFF_B),
])
df_raw = bq.query(sql, job_config=job_config).to_dataframe(create_bqstorage_client=False)

if TIMESTAMP_COL and TIMESTAMP_COL in df_raw.columns:
    df_raw = (df_raw
              .sort_values(KEY_COLUMNS + [STAFF_COL, TIMESTAMP_COL])
              .drop_duplicates(subset=KEY_COLUMNS + [STAFF_COL], keep="last")
              .reset_index(drop=True))

if df_raw.empty:
    raise SystemExit("No rows returned. Check STAFF_A/STAFF_B, keys, or source table.")

df_raw["__tag__"] = np.where(df_raw[STAFF_COL]==STAFF_A, "A",
                      np.where(df_raw[STAFF_COL]==STAFF_B, "B", "X"))
if (df_raw["__tag__"] == "X").any():
    raise ValueError("Unexpected staff IDs found outside STAFF_A/STAFF_B.")

wide = df_raw.pivot_table(index=KEY_COLUMNS, columns="__tag__", values=FIELDS, aggfunc="first")
wide.columns = [f"{c[0]}_{c[1]}" for c in wide.columns]
wide = wide.reset_index()

def case_label_from_row(row: pd.Series) -> str:
    return " | ".join(f"{k}={row[k]}" for k in KEY_COLUMNS)

CASE_INDEX: Dict[str, int] = {}
CASE_OPTIONS: List[str] = []
KEY_TUPLE_TO_LABEL: Dict[Tuple[str, ...], str] = {}
for i, r in wide.iterrows():
    lbl = case_label_from_row(r)
    CASE_INDEX[lbl] = i
    CASE_OPTIONS.append(lbl)
    KEY_TUPLE_TO_LABEL[tuple(str(r[k]) for k in KEY_COLUMNS)] = lbl
print(f"📦 Loaded {len(CASE_OPTIONS)} cases.\n")

# ─────────────────────────────────────────────────────────────────────────────
# Case-level match stats (NULL-safe)
# ─────────────────────────────────────────────────────────────────────────────

def compute_case_match_stats():
    # Build key list and "all keys NOT NULL" condition
    key_list = ", ".join([wrap_tick(k) for k in KEY_COLUMNS])
    not_null_cond = " AND ".join([f"{wrap_tick(k)} IS NOT NULL" for k in KEY_COLUMNS])

    # 1) Global counts from entire table in BigQuery (exclude NULL keys)
    sql = f"""
    WITH base AS (
      SELECT
        {key_list},
        COUNTIF({wrap_tick(STAFF_COL)} = @staffA) AS cA,
        COUNTIF({wrap_tick(STAFF_COL)} = @staffB) AS cB
      FROM `{TABLE_FQN}`
      WHERE {not_null_cond}
      GROUP BY {key_list}
    )
    SELECT
      COUNT(*)                             AS total_all_cases,
      COUNTIF(cA > 0)                      AS total_unique_A,
      COUNTIF(cB > 0)                      AS total_unique_B,
      COUNTIF(cA > 0 OR cB > 0)            AS total_in_A_or_B,
      COUNTIF(cA > 0 AND cB > 0)           AS total_both_A_and_B
    FROM base
    """

    job_cfg = bigquery.QueryJobConfig(query_parameters=[
        bigquery.ScalarQueryParameter("staffA", "STRING", STAFF_A),
        bigquery.ScalarQueryParameter("staffB", "STRING", STAFF_B),
    ])
    row = bq.query(sql, job_config=job_cfg).to_dataframe(
        create_bqstorage_client=False
    ).iloc[0]

    total_all_cases    = int(row["total_all_cases"])
    total_unique_A     = int(row["total_unique_A"])
    total_unique_B     = int(row["total_unique_B"])
    total_in_A_or_B    = int(row["total_in_A_or_B"])
    total_both_A_and_B = int(row["total_both_A_and_B"])

    # 2) Local: paired cases among df_raw (already STAFF_A/B filtered), also exclude NULL keys
    grouped = df_raw.groupby(KEY_COLUMNS)["__tag__"].agg(lambda s: set(s))

    local_paired_keys = []
    for key_vals, tags in grouped.items():
        # Skip groups with any NULL/NaN in key
        if isinstance(key_vals, tuple):
            if any(pd.isna(v) for v in key_vals):
                continue
            key_tuple = tuple(str(v) for v in key_vals)
        else:
            if pd.isna(key_vals):
                continue
            key_tuple = (str(key_vals),)

        if "A" in tags and "B" in tags:
            local_paired_keys.append(key_tuple)

    paired_with_diff = 0
    paired_all_match = 0

    for key_tuple in local_paired_keys:
        lbl = KEY_TUPLE_TO_LABEL.get(key_tuple)
        if lbl is None:
            continue  # safety

        idx = CASE_INDEX[lbl]
        r = wide.loc[idx]

        has_diff = False
        for f in FIELDS:
            a = r.get(f"{f}_A", np.nan)
            b = r.get(f"{f}_B", np.nan)
            if not values_equal(a, b, null_equals=NULL_EQUALS_MATCH):
                has_diff = True
                break

        if has_diff:
            paired_with_diff += 1
        else:
            paired_all_match += 1

    local_paired_count = paired_with_diff + paired_all_match

    # If still off, print a gentle note
    if total_both_A_and_B != local_paired_count:
        print(
            f"⚠️ Paired count mismatch: BigQuery={total_both_A_and_B}, local={local_paired_count}. "
            f"(check debug_paired_mismatch_nullsafe())"
        )

    return {
        "total_all_cases": total_all_cases,
        "total_unique_A": total_unique_A,
        "total_unique_B": total_unique_B,
        "total_in_A_or_B": total_in_A_or_B,
        "total_both_A_and_B": total_both_A_and_B,
        "paired_with_discrepancy": paired_with_diff,
        "paired_all_match": paired_all_match,
        "local_paired_count": local_paired_count,
    }

stats = compute_case_match_stats()

def _kpi(color, title, value, subtitle=""):
    return W.HTML(
        f"""
        <div style="
          padding:10px 14px;margin:4px;
          border-radius:12px;border:1px solid #e5e7eb;
          background:{color};min-width:220px">
          <div style="font-size:11px;color:#6b7280;text-transform:uppercase;letter-spacing:.08em">
            {title}
          </div>
          <div style="font-size:24px;font-weight:700;color:#111827;line-height:1.2">
            {value}
          </div>
          <div style="font-size:11px;color:#9ca3af;margin-top:2px">
            {subtitle}
          </div>
        </div>
        """
    )

def pct(part, total):
    return f"{(part/total*100):.1f}%" if total else "0.0%"

cards_row1 = W.HBox([
    _kpi("#f9fafb",
         "Total unique cases (all staff)",
         stats["total_all_cases"]),
    _kpi("#eef2ff",
         "Unique cases by A",
         stats["total_unique_A"]),
    _kpi("#eff6ff",
         "Unique cases by B",
         stats["total_unique_B"]),
    _kpi("#fff7ed",
         "Unique cases A or B",
         stats["total_in_A_or_B"],
         f"{pct(stats['total_in_A_or_B'], stats['total_all_cases'])} of all"),
])

cards_row2 = W.HBox([
    _kpi("#e0f2fe",
         "Unique cases A and B (paired)",
         stats["total_both_A_and_B"],
         f"{pct(stats['total_both_A_and_B'], stats['total_in_A_or_B'])} of A/B"),
    _kpi("#fef3c7",
         "Paired cases with discrepancy",
         stats["paired_with_discrepancy"],
         f"{pct(stats['paired_with_discrepancy'], stats['total_both_A_and_B'])} of paired"),
    _kpi("#ecfdf5",
         "Paired cases fully matched",
         stats["paired_all_match"],
         f"{pct(stats['paired_all_match'], stats['total_both_A_and_B'])} of paired"),
])

display(cards_row1)
display(cards_row2)



# ## 6) Match helpers
def normalize_scalar(x):
    if pd.isna(x): return np.nan
    if isinstance(x, str):
        s = x.strip()
        if s == "" or s.lower() in {"na","n/a","null","none"}: return np.nan
        try: return round(float(s), ROUND_DIGITS)
        except Exception: pass
        try: return pd.Timestamp(pd.to_datetime(s, errors="raise", utc=False)).round("s")
        except Exception: pass
        return s.lower()
    if isinstance(x, (int,float,np.integer,np.floating)): return round(float(x), ROUND_DIGITS)
    if isinstance(x, (pd.Timestamp,np.datetime64)):       return pd.Timestamp(x).round("s")
    return x

def values_equal(a, b, null_equals=True) -> bool:
    aa = normalize_scalar(a); bb = normalize_scalar(b)
    if pd.isna(aa) and pd.isna(bb): return bool(null_equals)
    if pd.isna(aa) or pd.isna(bb):  return False
    return aa == bb


# ## 7) Stores & progress (read from existing progress table)
ADJUDICATION_STORE: Dict[str, Dict[str, dict]] = {}
REQ_FIELDS = {}           # case_label -> set(fields required)
COMPLETED_FIELDS = {}     # case_label -> set(fields completed)
COMPLETE_CASES = set()    # done this session
PERSISTED_DONE = set()    # from adj_inc_progress

def load_progress():
    global PERSISTED_DONE
    try:
        df = bq.query(f"SELECT * FROM {fq(TBL_PROGRESS)}").result().to_dataframe()
    except Exception as e:
        raise SystemExit(f"❌ Cannot read {TBL_PROGRESS}: {e}")

    PERSISTED_DONE = set()
    if df is None or df.empty:
        return
    for _, r in df.iterrows():
        if str(r.get("status","")).lower() == "done":
            key_t = tuple(str(r.get(k, "")) for k in KEY_COLUMNS)
            lbl = KEY_TUPLE_TO_LABEL.get(key_t)
            if lbl: PERSISTED_DONE.add(lbl)

def mark_case_status(case_label: str, status: str, decided: int = None, required: int = None):
    idx = CASE_INDEX[case_label]
    src = wide.loc[idx]
    rec = {
        "status": status,
        "updated_at": now_utc(),          # TIMESTAMP object
        "adjudicator": ADJUDICATOR_NAME,
        "case_label": case_label,
        "decided": decided if decided is not None else None,   # numeric count
        "required": required if required is not None else None,# numeric count
        "staff_A": STAFF_A,
        "staff_B": STAFF_B,
        **{k: str(src[k]) for k in KEY_COLUMNS}
    }
    upsert_progress_row(rec)
    load_progress()


load_progress()


# ## 8) UI — toggles, panes, buttons
# ## 8) UI — toggles, panes, buttons

dd_case = W.Dropdown(options=[], description="Case ID", layout=W.Layout(width="60%"))
btn_prev = W.Button(description="⟵ Prev")
btn_next = W.Button(description="Next ⟶")

# outputs
out_table = W.Output(layout=W.Layout(border="1px solid #ddd",
                                     height=f"{MAX_TABLE_HEIGHT_PX}px",
                                     overflow="auto"))
out_form  = W.Output(layout=W.Layout())
#out_final = W.Output(layout=W.Layout())   # <- Final A/B/Adjudicator table

# top row: case selector + navigation
top_box = W.VBox([
    W.HBox([dd_case, W.Label("")]),
    W.HBox([btn_prev, btn_next])
])
display(top_box)
display(W.HTML("<hr>"))

# toggles
cb_cmp_only_diff = W.Checkbox(
    value=True,
    description="Comparison: only differences",
    indent=False
)
cb_adj_only_diff = W.Checkbox(
    value=True,
    description="Adjudication: only differences (affects required fields)",
    indent=False
)
cb_hide_done = W.Checkbox(
    value=HIDE_COMPLETED_DEFAULT,
    description="Hide completed cases",
    indent=False
)

display(W.HBox([cb_cmp_only_diff, cb_adj_only_diff, cb_hide_done]))

# panes
display(W.HTML("<b>Side-by-Side Comparison</b>"))
display(out_table)

display(W.HTML("<b>Adjudication (click A/B cell or type in Final)</b>"))
display(out_form)

#display(W.HTML("<hr><b>Final A / B / Adjudicator (this case)</b>"))
#display(out_final)

# action buttons
btn_save_case = W.Button(description="💾 Finalize Case (Mark Done → BQ)", button_style="success")
btn_reopen    = W.Button(description="🔓 Reopen Case", button_style="")
btn_save_all  = W.Button(description="⬆ Save All Completed → BigQuery", button_style="primary")

display(W.HTML("<hr>"))
display(W.HBox([btn_save_case, btn_reopen, btn_save_all]))



# ## 9) Required/completed logic & case list (fast; minimal recompute)
def _required_fields_for_label(label: str) -> set:
    idx = CASE_INDEX[label]
    if cb_adj_only_diff.value:
        return {
            f for f in FIELDS
            if not values_equal(
                wide.loc[idx, f"{f}_A"] if f"{f}_A" in wide.columns else np.nan,
                wide.loc[idx, f"{f}_B"] if f"{f}_B" in wide.columns else np.nan,
                null_equals=NULL_EQUALS_MATCH
            )
        }
    else:
        return set(FIELDS)

def compute_required_fields_for_all_cases():
    REQ_FIELDS.clear(); COMPLETED_FIELDS.clear(); COMPLETE_CASES.clear()
    for label in CASE_OPTIONS:
        idx = CASE_INDEX[label]
        req = _required_fields_for_label(label)
        REQ_FIELDS[label] = req
        decided = set()
        for f in req:
            a = wide.loc[idx, f"{f}_A"] if f"{f}_A" in wide.columns else np.nan
            b = wide.loc[idx, f"{f}_B"] if f"{f}_B" in wide.columns else np.nan
            if values_equal(a, b, null_equals=NULL_EQUALS_MATCH):
                decided.add(f); continue
            rec = ADJUDICATION_STORE.get(label, {}).get(f)
            if rec and (rec.get("choice") in {"A","B"} or (rec.get("choice")=="Other" and str(rec.get("final") or "").strip()!="")):
                decided.add(f)
        COMPLETED_FIELDS[label] = decided
        if len(decided) == len(req): COMPLETE_CASES.add(label)

def missing_fields_for_case(case_label: str) -> list:
    return sorted(REQ_FIELDS.get(case_label, set()) - COMPLETED_FIELDS.get(case_label, set()))

def case_is_complete(case_label: str) -> bool:
    return len(REQ_FIELDS.get(case_label, set())) == len(COMPLETED_FIELDS.get(case_label, set()))

def filtered_case_options() -> List[str]:
    if cb_hide_done.value:
        hidden = set(PERSISTED_DONE) | set(COMPLETE_CASES)
        return [lbl for lbl in CASE_OPTIONS if lbl not in hidden]
    return list(CASE_OPTIONS)

def refresh_case_dropdown(preserve_current=True):
    opts = filtered_case_options()
    cur = dd_case.value
    dd_case.options = opts
    if preserve_current and cur in opts:
        dd_case.value = cur
    elif opts:
        dd_case.value = opts[0]
    else:
        dd_case.value = None

def update_save_buttons_state():
    cur = dd_case.value if 'dd_case' in globals() else None
    btn_save_case.disabled = (not cur) or (not case_is_complete(cur))
    btn_save_all.disabled  = (len(COMPLETE_CASES) == 0)
    btn_reopen.disabled    = (not cur) or (cur not in PERSISTED_DONE)

compute_required_fields_for_all_cases()
refresh_case_dropdown(preserve_current=False)



# ## 10) Rendering


def table_A_B_Adjudicator(case_label: str) -> pd.DataFrame:
    """
    Build a clean A/B/Adjudicator table for the current case.

    - Columns: field | A_entry | B_entry | Adjudicated
    - Skips rows where A, B, and final are all empty.
    - If A & B match (per values_equal) and no explicit adjudication:
        - final = that matched value.
    - If cb_adj_only_diff is True:
        - show only fields with discrepancies OR with an explicit adjudicated value
          that differs from both original entries.
    """
    if not case_label:
        return pd.DataFrame(columns=["field", "A_entry", "B_entry", "Adjudicated"])

    show_diff_only = cb_adj_only_diff.value  # tie behavior to your adjudication toggle

    idx = CASE_INDEX[case_label]
    row = wide.loc[idx]

    rows = []
    for f in FIELDS:
        a = row.get(f"{f}_A", np.nan)
        b = row.get(f"{f}_B", np.nan)

        rec = ADJUDICATION_STORE.get(case_label, {}).get(f)
        if rec is not None:
            final = rec.get("final")
        else:
            # auto-fill final when A & B agree and not both blank
            if values_equal(a, b, null_equals=NULL_EQUALS_MATCH) and not (pd.isna(a) and pd.isna(b)):
                final = a
            else:
                final = None

        # normalize empties
        def is_empty(v):
            return v is None or (isinstance(v, float) and pd.isna(v)) or str(v).strip() == ""

        a_empty = is_empty(a)
        b_empty = is_empty(b)
        final_empty = is_empty(final)

        # skip if absolutely nothing there
        if a_empty and b_empty and final_empty:
            continue

        equal_ab = values_equal(a, b, null_equals=NULL_EQUALS_MATCH)

        # If showing only differences:
        # - drop pure matches where final is just the same as A/B and no custom change
        if show_diff_only:
            # detect if final is effectively just the agreed value
            if equal_ab and (final_empty or values_equal(final, a, null_equals=True)):
                continue

        rows.append((
            f,
            None if a_empty else a,
            None if b_empty else b,
            None if final_empty else final
        ))

    return pd.DataFrame(rows, columns=["field", "A_entry", "B_entry", "Adjudicated"])


def refresh_final_table(case_label: str):
    out_final.clear_output()
    if not case_label:
        return

    df_abj = table_A_B_Adjudicator(case_label)

    with out_final:
        if df_abj.empty:
            display(HTML("<i>No differing/adjudicated fields to display for this case.</i>"))
        else:
            styler = (
                df_abj
                .style
                .hide(axis="index")
                .set_properties(**{
                    "font-size": "12px",
                    "white-space": "nowrap"
                })
            )
            display(HTML(styler.to_html()))

def render_case(case_label: str):
    out_table.clear_output()
    out_form.clear_output()
    if not case_label:
        return

    # init required/completed sets for this case
    if case_label not in REQ_FIELDS:
        REQ_FIELDS[case_label] = _required_fields_for_label(case_label)
        COMPLETED_FIELDS.setdefault(case_label, set())

    idx = CASE_INDEX[case_label]
    row = wide.loc[idx]

    # ---- 1) Build comparison table (A vs B) ----
    data = []
    for f in FIELDS:
        a = row.get(f"{f}_A", np.nan)
        b = row.get(f"{f}_B", np.nan)
        match = values_equal(a, b, null_equals=NULL_EQUALS_MATCH)
        data.append((f, a, b, "Yes" if match else "No"))
    view_df = pd.DataFrame(data, columns=["field", "A", "B", "Match?"]).set_index("field")

    # filter by "comparison only diff" toggle
    if cb_cmp_only_diff.value:
        diff_idx = [f for f in FIELDS if view_df.loc[f, "Match?"] == "No"]
        cmp_df = view_df.loc[diff_idx] if diff_idx else view_df.iloc[0:0]
    else:
        cmp_df = view_df.loc[FIELDS] if set(FIELDS).issubset(view_df.index) else view_df

    # style differences
    def color_cell(val_a, val_b):
        a_isna, b_isna = pd.isna(val_a), pd.isna(val_b)
        if a_isna and b_isna:
            return ("", "")
        if a_isna ^ b_isna:
            return ('background-color:#fff0b3;', 'background-color:#fff0b3;')
        if not values_equal(val_a, val_b, null_equals=False):
            return ('background-color:#ffd6d6;', 'background-color:#ffd6d6;')
        return ("", "")

    styles_a, styles_b = [], []
    for f in cmp_df.index:
        sa, sb = color_cell(cmp_df.loc[f, "A"], cmp_df.loc[f, "B"])
        styles_a.append(sa)
        styles_b.append(sb)

    styler = (
        cmp_df.style
        .set_properties(**{"font-size": "12px"})
        .apply(lambda s: styles_a, subset=pd.IndexSlice[:, ["A"]])
        .apply(lambda s: styles_b, subset=pd.IndexSlice[:, ["B"]])
    )

    with out_table:
        display(HTML(styler.to_html()))

    # ---- 2) Pre-store exact matches (auto "Match") ----
    for f in FIELDS:
        a = view_df.loc[f, "A"]
        b = view_df.loc[f, "B"]
        if values_equal(a, b, null_equals=NULL_EQUALS_MATCH):
            ADJUDICATION_STORE.setdefault(case_label, {}).setdefault(
                f,
                {
                    "choice": "Match",
                    "final": a if not pd.isna(a) else b,
                    "a": a,
                    "b": b,
                },
            )

    # ---- 3) Build adjudication widgets (A/B/Other) ----
    target_fields = (
        sorted(REQ_FIELDS.get(case_label, set()))
        if cb_adj_only_diff.value
        else sorted(FIELDS)
    )

    rows = []
    case_store = ADJUDICATION_STORE.get(case_label, {})

    for f in target_fields:
        a = view_df.loc[f, "A"]
        b = view_df.loc[f, "B"]
        is_match = values_equal(a, b, null_equals=NULL_EQUALS_MATCH)

        prior = case_store.get(f, {})
        chosen = prior.get("choice")
        final_v = prior.get("final")

        a_disp = "" if pd.isna(a) else str(a)
        b_disp = "" if pd.isna(b) else str(b)

        btnA = W.Button(
            description=a_disp if len(a_disp) <= 40 else a_disp[:37] + "…",
            layout=W.Layout(width="260px"),
            tooltip=a_disp or "(blank)",
        )
        btnB = W.Button(
            description=b_disp if len(b_disp) <= 40 else b_disp[:37] + "…",
            layout=W.Layout(width="260px"),
            tooltip=b_disp or "(blank)",
        )

        def refresh_styles(xA, xB, sel):
            xA.button_style = "success" if sel == "A" else ""
            xB.button_style = "success" if sel == "B" else ""

        txtFinal = W.Text(
            value="" if final_v is None or chosen != "Other" else str(final_v),
            placeholder="Type corrected value…",
            layout=W.Layout(width="280px"),
        )

        refresh_styles(btnA, btnB, chosen)

        def on_click_a(
            _,
            field=f,
            a_val=a,
            b_val=b,
            this_case=case_label,
            xA=btnA,
            xB=btnB,
        ):
            ADJUDICATION_STORE.setdefault(this_case, {})[field] = {
                "choice": "A",
                "final": a_val,
                "a": a_val,
                "b": b_val,
            }
            if field in REQ_FIELDS.get(this_case, set()):
                COMPLETED_FIELDS.setdefault(this_case, set()).add(field)
                if len(COMPLETED_FIELDS[this_case]) == len(REQ_FIELDS[this_case]):
                    COMPLETE_CASES.add(this_case)
                else:
                    COMPLETE_CASES.discard(this_case)
            refresh_styles(xA, xB, "A")
            update_save_buttons_state()
            refresh_final_table(this_case)

        def on_click_b(
            _,
            field=f,
            a_val=a,
            b_val=b,
            this_case=case_label,
            xA=btnA,
            xB=btnB,
        ):
            ADJUDICATION_STORE.setdefault(this_case, {})[field] = {
                "choice": "B",
                "final": b_val,
                "a": a_val,
                "b": b_val,
            }
            if field in REQ_FIELDS.get(this_case, set()):
                COMPLETED_FIELDS.setdefault(this_case, set()).add(field)
                if len(COMPLETED_FIELDS[this_case]) == len(REQ_FIELDS[this_case]):
                    COMPLETE_CASES.add(this_case)
                else:
                    COMPLETE_CASES.discard(this_case)
            refresh_styles(xA, xB, "B")
            update_save_buttons_state()
            refresh_final_table(this_case)

        def on_text(
            change,
            field=f,
            a_val=a,
            b_val=b,
            this_case=case_label,
            xA=btnA,
            xB=btnB,
            _state={"had_text": bool(str(final_v or "").strip())},
        ):
            if change["name"] != "value":
                return
            val = str(change["new"] or "").strip()
            if not val:
                return
            ADJUDICATION_STORE.setdefault(this_case, {})[field] = {
                "choice": "Other",
                "final": val,
                "a": a_val,
                "b": b_val,
            }
            if field in REQ_FIELDS.get(this_case, set()):
                COMPLETED_FIELDS.setdefault(this_case, set()).add(field)
                if len(COMPLETED_FIELDS[this_case]) == len(REQ_FIELDS[this_case]):
                    COMPLETE_CASES.add(this_case)
                else:
                    COMPLETE_CASES.discard(this_case)
            refresh_styles(xA, xB, None)
            if not _state["had_text"]:
                _state["had_text"] = True
                update_save_buttons_state()
            refresh_final_table(this_case)

        btnA.on_click(on_click_a)
        btnB.on_click(on_click_b)
        txtFinal.observe(on_text)

        rows.append(
            W.HBox(
                [
                    W.Label(f, layout=W.Layout(width="220px")),
                    btnA,
                    btnB,
                    W.Label("Yes" if is_match else "No",
                            layout=W.Layout(width="80px")),
                    txtFinal,
                ]
            )
        )

    # ---- 4) Render adjudication widgets ----
    with out_form:
        header = W.HBox(
            [
                W.HTML("<b style='display:inline-block;width:220px'>Field</b>"),
                W.HTML("<b style='display:inline-block;width:260px'>A (click to choose)</b>"),
                W.HTML("<b style='display:inline-block;width:260px'>B (click to choose)</b>"),
                W.HTML("<b style='display:inline-block;width:80px'>Match?</b>"),
                W.HTML("<b style='display:inline-block;width:280px'>Final (typing = Other)</b>"),
            ]
        )
        display(W.VBox([header] + rows))

    # ---- 5) Show current A/B/Adjudicator table & refresh buttons ----
    refresh_final_table(case_label)
    update_save_buttons_state()



# ## 11) Build dataframes for BigQuery writes
def build_long_df_for_cases(labels: List[str]) -> pd.DataFrame:
    rows_long = []
    now_ts = pd.Timestamp.utcnow().to_pydatetime()
    for case_label in labels:
        idx = CASE_INDEX[case_label]
        base = {k: str(wide.loc[idx, k]) for k in KEY_COLUMNS}
        store = ADJUDICATION_STORE.get(case_label, {})
        for f in FIELDS:
            a = wide.loc[idx, f"{f}_A"] if f"{f}_A" in wide.columns else np.nan
            b = wide.loc[idx, f"{f}_B"] if f"{f}_B" in wide.columns else np.nan
            if not values_equal(a, b, null_equals=NULL_EQUALS_MATCH):
                rec = store.get(f, {})
                choice = rec.get("choice")
                final_value = rec.get("final")
            else:
                choice = "Match"
                final_value = a if not pd.isna(a) else b
            rows_long.append({
                **base,
                "field": f,
                "A_value": "" if pd.isna(a) else str(a),
                "B_value": "" if pd.isna(b) else str(b),
                "choice": "" if choice is None else str(choice),
                "final_value": "" if final_value is None else str(final_value),
                "case_label": case_label,
                "staff_A": STAFF_A,
                "staff_B": STAFF_B,
                "generated_at": now_ts
            })
    return pd.DataFrame(rows_long)

def build_wide_df_for_cases(labels: List[str]) -> pd.DataFrame:
    wide_records = {}
    now_ts = pd.Timestamp.utcnow().to_pydatetime()
    for case_label in labels:
        idx = CASE_INDEX[case_label]
        d = {k: str(wide.loc[idx, k]) for k in KEY_COLUMNS}
        d["staff_A"] = STAFF_A
        d["staff_B"] = STAFF_B
        d["generated_at"] = now_ts
        store = ADJUDICATION_STORE.get(case_label, {})
        for f in FIELDS:
            a = wide.loc[idx, f"{f}_A"] if f"{f}_A" in wide.columns else np.nan
            b = wide.loc[idx, f"{f}_B"] if f"{f}_B" in wide.columns else np.nan
            if not values_equal(a, b, null_equals=NULL_EQUALS_MATCH):
                rec = store.get(f, {})
                final_value = rec.get("final")
            else:
                final_value = a if not pd.isna(a) else b
            d[f] = "" if final_value is None or (isinstance(final_value,float) and pd.isna(final_value)) else str(final_value)
        wide_records[tuple(d[k] for k in KEY_COLUMNS)] = d
    return pd.DataFrame(list(wide_records.values())) if wide_records else pd.DataFrame()

def build_summary_df(num_cases: int, total_rows: int) -> pd.DataFrame:
    return pd.DataFrame([{
        "project_id": PROJECT_ID,
        "dataset_id": DATASET_ID,
        "table_id": TABLE_ID,
        "staff_A": STAFF_A,
        "staff_B": STAFF_B,
        "keys": ", ".join(KEY_COLUMNS),
        "fields_compared": len(FIELDS),
        "cases_adjudicated": int(num_cases),
        "total_rows_saved": int(total_rows),
        "generated_at": now_utc(),   # << was strftime, now datetime
    }])



# ## 12) Handlers — finalize, reopen, save all (→ BigQuery)
def finalize_current_case(*args):
    case_label = dd_case.value
    if not case_label: return
    miss = missing_fields_for_case(case_label)
    if miss:
        print(f"⚠ You must adjudicate all required fields before finalizing.\nMissing ({len(miss)}): {', '.join(miss[:12])}{'…' if len(miss)>12 else ''}")
        update_save_buttons_state(); return

    decided  = len(COMPLETED_FIELDS.get(case_label, set()))
    required = len(REQ_FIELDS.get(case_label, set()))
    mark_case_status(case_label, "done", decided=decided, required=required)

    if SAVE_ON_FINALIZE:
        df_long = build_long_df_for_cases([case_label])
        df_wide = build_wide_df_for_cases([case_label])
        write_long_df_upsert(df_long)
        write_wide_df_upsert(df_wide)
        write_summary_append(build_summary_df(1, df_long.shape[0]))
        print(f"✅ Finalized & saved to BigQuery: {case_label}")
    else:
        print(f"✅ Finalized: {case_label} (hidden if 'Hide completed' is checked)")

    COMPLETE_CASES.add(case_label)
    refresh_case_dropdown(preserve_current=False)
    if dd_case.value: render_case(dd_case.value)
    update_save_buttons_state()

def reopen_current_case(*args):
    case_label = dd_case.value
    if not case_label: return
    if case_label not in PERSISTED_DONE:
        print("ℹ This case is not marked DONE in progress; nothing to reopen.")
        return

    decided  = len(COMPLETED_FIELDS.get(case_label, set()))
    required = len(REQ_FIELDS.get(case_label, set()))
    mark_case_status(case_label, "open", decided=decided, required=required)
    COMPLETE_CASES.discard(case_label)

    # Prefill from adj_inc_long if available
    try:
        where_keys = " AND ".join([f"{k}=@{k}" for k in KEY_COLUMNS])
        params = [bigquery.ScalarQueryParameter(k, "STRING", str(wide.loc[CASE_INDEX[case_label], k])) for k in KEY_COLUMNS]
        df_prev = bq.query(f"SELECT field, final_value FROM {fq(TBL_LONG)} WHERE {where_keys}",
                           job_config=bigquery.QueryJobConfig(query_parameters=params)).to_dataframe()
        if df_prev is not None and not df_prev.empty:
            store = ADJUDICATION_STORE.setdefault(case_label, {})
            idx = CASE_INDEX[case_label]
            for _, rr in df_prev.iterrows():
                f = str(rr["field"])
                if f in FIELDS:
                    a = wide.loc[idx, f"{f}_A"] if f"{f}_A" in wide.columns else np.nan
                    b = wide.loc[idx, f"{f}_B"] if f"{f}_B" in wide.columns else np.nan
                    v = rr["final_value"]
                    if v is not None and str(v) != "":
                        store[f] = {"choice":"Other","final":str(v),"a":a,"b":b}
            # recompute completion
            req = REQ_FIELDS.get(case_label,set())
            done = set()
            for f in req:
                rec = store.get(f)
                if rec and (rec.get("choice") in {"A","B"} or (rec.get("choice")=="Other" and str(rec.get("final") or "").strip()!="")):
                    done.add(f)
                else:
                    a = wide.loc[idx, f"{f}_A"] if f"{f}_A" in wide.columns else np.nan
                    b = wide.loc[idx, f"{f}_B"] if f"{f}_B" in wide.columns else np.nan
                    if values_equal(a,b,null_equals=NULL_EQUALS_MATCH): done.add(f)
            COMPLETED_FIELDS[case_label] = done
    except Exception:
        pass

    refresh_case_dropdown(preserve_current=True)
    if dd_case.value: render_case(dd_case.value)
    update_save_buttons_state()
    print(f"🔓 Reopened case: {case_label}")

def save_all_to_bigquery(*args):
    complete_cases = [lbl for lbl in CASE_OPTIONS if case_is_complete(lbl)]
    if not complete_cases:
        print("⚠ No completed cases to save yet.")
        return
    df_long = build_long_df_for_cases(complete_cases)
    df_wide = build_wide_df_for_cases(complete_cases)
    write_long_df_upsert(df_long)
    write_wide_df_upsert(df_wide)
    write_summary_append(build_summary_df(len(complete_cases), df_long.shape[0]))
    print(f"✅ Saved to BigQuery: {len(complete_cases)} case(s) → {TBL_LONG}, {TBL_WIDE}, {TBL_SUMMARY}")

# Bind handlers
btn_save_case.on_click(finalize_current_case)
btn_reopen.on_click(reopen_current_case)
btn_save_all.on_click(save_all_to_bigquery)

def on_case_change(change):
    if change["name"]=="value" and change["new"]:
        ADJUDICATION_STORE.setdefault(change["new"], {})
        render_case(change["new"]); update_save_buttons_state()

dd_case.observe(on_case_change)

def on_toggle_cmp(change):
    if change["name"]=="value" and dd_case.value:
        render_case(dd_case.value)

def on_toggle_adj(change):
    if change["name"]=="value":
        compute_required_fields_for_all_cases()
        refresh_case_dropdown(preserve_current=True)
        if dd_case.value: render_case(dd_case.value)
        update_save_buttons_state()

def on_toggle_hide_done(change):
    if change["name"]=="value":
        refresh_case_dropdown(preserve_current=False)
        if dd_case.value: render_case(dd_case.value)
        update_save_buttons_state()

cb_cmp_only_diff.observe(on_toggle_cmp)
cb_adj_only_diff.observe(on_toggle_adj)
cb_hide_done.observe(on_toggle_hide_done)

# Initial render
refresh_case_dropdown(preserve_current=False)
if dd_case.options:
    dd_case.value = dd_case.options[0]
    render_case(dd_case.options[0])
    update_save_buttons_state()
else:
    print("No cases available.")

